In [1]:
from math import exp, log, sqrt

In [2]:
def TridiagonalSolver(diag, upper, lower, rhs):
    size = len(rhs)
    for i in range(size-1):
        diag[i+1] = diag[i+1] - lower[i+1] / diag[i] * upper[i]
        rhs[i+1]  = rhs[i+1] - lower[i+1] / diag[i] * rhs[i]

    rhs[size-1] = rhs[size-1] / diag[size-1]
    for i in range(size-1):
        j = size - 2 - i
        rhs[j] = (rhs[j] - upper[j] * rhs[j+1]) / diag[j]

In [3]:
diag = [-2.6, -2.6, -2.6, -2.6]
upper = [1., 1., 1., 1.]
lower = [1., 1., 1., 1.]
rhs = [-240., 0., 0., -150.]
TridiagonalSolver(diag, upper, lower, rhs)
print(rhs)

[118.11216764581187, 67.09163587911088, 56.32608563987643, 79.35618678456785]


In [4]:
def OneDimBackwardPDEOptionPricer(spot,
                                  repo,
                                  rate,
                                  volatility,
                                  expiry,
                                  payoff,
                                  number_of_time_steps,
                                  number_of_state_variables,
                                  number_of_standard_deviations,
                                  is_american = False,
                                  remove_funding = False,
                                  remove_discounting = False,
                                  early_exercise_policy = None):
    # Calculate terms appearing in the PDE
    temp       = .5 * volatility * volatility
    convection = repo - temp
    diffusion  = temp
    source     = rate
    
    # Forward and discount curves
    # Assuming state variables x = log(S(t)/S(0)),
    # and evolving V(t,x)
    forward  = lambda t : spot
    discount = lambda t : 1.
    
    # If using x(t) = log(S(t)/F(t)) as state variables
    if remove_funding:
        convection = -temp
        forward    = lambda t : spot * exp(repo * t)
        
    # If evolving e^{-rt}V(t,x)
    if remove_discounting:
        source   = 0.
        discount = lambda t : exp(-rate * t)
        
    # Set up finite difference grid
    # Temporal direction
    time_step = -expiry / number_of_time_steps # Time step is negative, while equation is evolving forward
    times = []
    for i in range(number_of_time_steps):
        times.append(expiry - expiry * (i+1) / number_of_time_steps)
    
    # Spatial direction
    standard_deviation = volatility * sqrt(expiry)
    if number_of_state_variables % 2 == 0:
        raise ValueError("Number of state variables must be odd")
    
    central_index = number_of_state_variables // 2
    state_variable_step = number_of_standard_deviations * standard_deviation / central_index
    state_variables = [0.] * number_of_state_variables
    for i in range(number_of_state_variables):
        state_variables[i] = (i - central_index) * state_variable_step
        
    # Now, we are ready for backward induction
    # First, set up initial condition
    exp_state_variables = [exp(sv) for sv in state_variables]
    prices = [discount(expiry) * payoff(forward(expiry) * esv) for esv in exp_state_variables]
    
    # Evolve with the implicit scheme
    dt_over_dx = time_step / state_variable_step
    dt_over_dx_dx = dt_over_dx / state_variable_step
    k = number_of_state_variables-1
        
    diag  = [0.] * number_of_state_variables
    upper = [0.] * number_of_state_variables
    lower = [0.] * number_of_state_variables
    for time in times:
        for j in range(number_of_state_variables):
            diag[j]  = 1. - source * time_step - 2. * diffusion * dt_over_dx_dx;
            upper[j] =   .5 * convection * dt_over_dx + diffusion * dt_over_dx_dx;
            lower[j] = - .5 * convection * dt_over_dx + diffusion * dt_over_dx_dx;
            
        # Apply boundary conditions -- does not matter that much
        prices[0] = prices[0] - lower[0] * payoff(forward(time) * exp_state_variables[0])
        prices[k] = prices[k] - upper[k] * payoff(forward(time) * exp_state_variables[k])
        
        
        if is_american:
            TridiagonalSolver(diag, upper, lower, prices)
            exercises = [discount(time) * payoff(forward(time) * esv) for esv in exp_state_variables]
            prices = [exercise if exercise > price else price for (price, exercise) in zip(prices, exercises) ]
        else:
            TridiagonalSolver(diag, upper, lower, prices)
        
    return prices[central_index]

In [5]:
spot = 100.
repo = .02
rate = .04
volatility = .2
expiry = 1.
number_of_time_steps = 700
number_of_state_variables = 1301
number_of_standard_deviations = 6.5

In [6]:
# Put option
diff = lambda x : spot - x
payoff = lambda x : diff(x) if diff(x) > 0. else 0.

In [7]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False,
                                    True))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True,
                                    True))

6.796979690847332
6.7971437795454985
6.796887617269901
6.797098831156888


In [8]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False,
                                    True))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True,
                                    True))

7.01501986233488
7.015197028366577
7.0150604431917785
7.015274996617345


In [9]:
# Call option
diff = lambda x : x - spot
payoff = lambda x : diff(x) if diff(x) > 0. else 0.

In [10]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    False,
                                    True))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    False, 
                                    True,
                                    True))

8.737821964429392
8.738095855358432
8.737812597347801
8.738021593101958


In [11]:
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    False,
                                    True))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True, 
                                    False))
print(OneDimBackwardPDEOptionPricer(spot, 
                                    repo, 
                                    rate, 
                                    volatility, 
                                    expiry, 
                                    payoff, 
                                    number_of_time_steps, 
                                    number_of_state_variables, 
                                    number_of_standard_deviations, 
                                    True, 
                                    True,
                                    True))

8.737827018690215
8.738100907937273
8.737817623692191
8.738026624340675
